In [7]:
# Import necessary libraries
import os
import re
from docx import Document
import subprocess
import datetime


In [8]:
# Define file paths using relative paths
# Assumes the notebook is in a parent directory, and 'prompt_templates' and 'knowledge_base' are subdirectories
PROMPT_TEMPLATES_DIR = 'prompt_templates/Prompt Templates/'
KNOWLEDGE_BASE_DIR = 'knowledge_base/Knowledge Base/'

--- Helper Functions ---

In [9]:
def load_prompt(template_name):
    # Construct path relative to the script/notebook's directory
    # os.path.dirname(__file__) might not work as expected in a notebook directly executed cell
    # So, we rely on the CWD being the notebook's directory or the user setting up paths correctly.
    # For simplicity with relative paths, we assume the notebook is run from its containing folder.
    filepath = os.path.join(PROMPT_TEMPLATES_DIR, template_name + ".docx")
    try:
        if not os.path.exists(filepath):
            print(f"Prompt file not found: {filepath} (Full path: {os.path.abspath(filepath)})")
            return f"[Prompt for {template_name} not found]"
        doc = Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs if para.text.strip()])
    except Exception as e:
        print(f"Error loading prompt {template_name} from {filepath}: {e}")
        return f"[Error loading prompt: {template_name}]"

In [10]:
def extract_text_from_pdf(pdf_path):
    try:
        # Ensure pdf_path is correctly formed if KNOWLEDGE_BASE_DIR is relative
        full_pdf_path = os.path.abspath(pdf_path)
        if not os.path.exists(full_pdf_path):
            print(f"PDF file not found for extraction: {full_pdf_path}")
            return ""
        result = subprocess.run(['pdftotext', full_pdf_path, '-'], capture_output=True, text=True, check=True, encoding='utf-8', errors='ignore')
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"Error extracting text from {os.path.abspath(pdf_path)} with pdftotext: {e}")
        return ""
    except FileNotFoundError:
        print(f"pdftotext command not found. Please ensure poppler-utils is installed.")
        return ""

In [11]:
def load_knowledge_base():
    knowledge = {}
    # Check if the relative KNOWLEDGE_BASE_DIR exists
    if not os.path.isdir(KNOWLEDGE_BASE_DIR):
        print(f"Knowledge base directory not found: {KNOWLEDGE_BASE_DIR} (Full path: {os.path.abspath(KNOWLEDGE_BASE_DIR)})")
        print("Please ensure you have a 'knowledge_base/Knowledge Base/' subdirectory with your PDF files relative to where the notebook is run.")
        return knowledge
        
    for filename in os.listdir(KNOWLEDGE_BASE_DIR):
        if filename.endswith('.pdf'):
            filepath = os.path.join(KNOWLEDGE_BASE_DIR, filename)
            key_name = filename.replace(' _ Barbeque Nation.pdf', '')
            key_name = re.sub(r'[^a-zA-Z0-9_]', '_', key_name)
            key_name = key_name.replace('New_Delhi_CP_cp', 'New_Delhi_Connaught_Place')
            knowledge[key_name] = extract_text_from_pdf(filepath) # extract_text_from_pdf now handles abspath
    return knowledge

In [12]:
# --- Chatbot State Machine ---
class BarbiqnationChatbot:
    def __init__(self):
        self.state = 'START'
        self.context = {'booking_details': {}}
        self.prompts = {
            'collect_city': load_prompt('collect_city'),
            'collect_contact_information': load_prompt('collect_contact_information'),
            'master_collect_generic': load_prompt('master_collect'),
            'master_inform_greeting': load_prompt('master_inform') or "Welcome to Barbeque Nation! How can I help you today? You can make a new booking, cancel/modify an existing one, or ask a question.",
            'master_inform_fallback	': "I'm sorry, I didn't understand that. Can you please rephrase?",
            'ask_location_bengaluru': "Great! Which location in Bengaluru? (e.g., Indiranagar, Electronic City, JP Nagar, Koramangala)",
            'ask_location_delhi': "Great! Which location in Delhi? (e.g., Connaught Place, Vasant Kunj, Janakpuri)",
            'ask_datetime': "Okay. For what date and time would you like to make the booking? (e.g., Tomorrow 7 PM, 25th May 1 PM)",
            'ask_pax_size': "How many guests will be joining? (e.g., 2 people, 5)",
            'confirm_booking_prompt': "Just to confirm, you want to book for {pax_size} at Barbeque Nation {location} in {city} on {date} at {time}. Is that correct? (Yes/No)",
            'booking_confirmed': "Your booking is confirmed! We look forward to seeing you.",
            'booking_cancelled_user': "Okay, I will not proceed with the booking.",
            'ask_cancel_modify_details': "Okay, to cancel or modify, I need some details. What is the date and time of your booking, and the name under which it was made?",
            'confirm_cancellation_prompt': "So you want to cancel the booking under the name {name} for {datetime_str}? (Yes/No)",
            'cancellation_confirmed': "Your booking has been cancelled.",
            'cancellation_aborted': "Okay, I will not cancel the booking.",
            'ask_enquiry': "Sure, what is your question?",
            'feedback_acknowledgement': "Thank you for your feedback/complaint. We will look into this."
        }
        self.knowledge_base = load_knowledge_base()
        print(f"Knowledge base loaded. Keys: {list(self.knowledge_base.keys())}")
        if not self.knowledge_base:
             print("Warning: Knowledge base is empty. Ensure PDF files are in the correct relative path: 'knowledge_base/Knowledge Base/'")

    def get_prompt(self, key, **kwargs):
        # Corrected fallback key
        prompt_template = self.prompts.get(key, self.prompts.get('master_inform_fallback	', "Sorry, I couldn't understand."))
        try:
            return prompt_template.format(**kwargs)
        except KeyError as e:
            print(f"Warning: Missing key {e} for prompt {key}")
            return prompt_template

    def handle_message(self, message):
        message_lower = message.lower()
        response = ""
        bd = self.context.get('booking_details', {})

        if "start over".lower() in message_lower or "restart".lower() in message_lower:
            self.state = "START"
            self.context = {'booking_details': {}}
            return self.get_prompt('master_inform_greeting')

        if self.state == 'START':
            response = self.get_prompt('master_inform_greeting')
            self.state = 'AWAITING_INITIAL_CHOICE'

        elif self.state == 'AWAITING_INITIAL_CHOICE':
            if any(kw in message_lower for kw in ['new booking', 'book a table', 'make a reservation', 'book']):
                self.state = 'COLLECT_CITY'
                response = self.get_prompt('collect_city')
            elif any(kw in message_lower for kw in ['cancel', 'modify', 'change booking']):
                self.state = 'COLLECT_CANCEL_MODIFY_DATETIME'
                response = "Okay, for which date and time is the booking you want to cancel or modify?"
            elif any(kw in message_lower for kw in ['question', 'enquiry', 'ask', 'info']):
                self.state = 'HANDLE_ENQUIRY_ASK'
                response = self.get_prompt('ask_enquiry')
            elif any(kw in message_lower for kw in ['feedback', 'complaint']):
                self.state = 'HANDLE_FEEDBACK'
                response = self.get_prompt('feedback_acknowledgement')
                self.state = 'AWAITING_INITIAL_CHOICE'
            else:
                response = self.get_prompt('master_inform_fallback	') # Corrected key

        elif self.state == 'COLLECT_CITY':
            if any(city_kw in message_lower for city_kw in ['bengaluru', 'bangalore']):
                bd['city'] = 'Bengaluru'
                self.state = 'COLLECT_LOCATION_BENGALURU'
                response = self.get_prompt('ask_location_bengaluru')
            elif 'delhi' in message_lower:
                bd['city'] = 'Delhi'
                self.state = 'COLLECT_LOCATION_DELHI'
                response = self.get_prompt('ask_location_delhi')
            else:
                response = 'Sorry, we only have outlets in Bengaluru and Delhi. Please choose one.'

        elif self.state == 'COLLECT_LOCATION_BENGALURU':
            bd['location'] = message
            self.state = 'COLLECT_DATETIME'
            response = self.get_prompt('ask_datetime')

        elif self.state == 'COLLECT_LOCATION_DELHI':
            bd['location'] = message
            self.state = 'COLLECT_DATETIME'
            response = self.get_prompt('ask_datetime')

        elif self.state == 'COLLECT_DATETIME':
            bd['datetime_str'] = message
            parts = message.split()
            bd['date'] = parts[0] if len(parts) > 0 else 'Not specified'
            bd['time'] = ' '.join(parts[1:]) if len(parts) > 1 else 'Not specified'
            self.state = 'COLLECT_PAX_SIZE'
            response = self.get_prompt('ask_pax_size')

        elif self.state == 'COLLECT_PAX_SIZE':
            match = re.search(r'\d+', message)
            if match:
                bd['pax_size'] = match.group(0)
            else:
                bd['pax_size'] = message
            self.state = 'CONFIRM_BOOKING'
            response = self.get_prompt('confirm_booking_prompt', city=bd.get('city'), location=bd.get('location'), date=bd.get('date'), time=bd.get('time'), pax_size=bd.get('pax_size'))

        elif self.state == 'CONFIRM_BOOKING':
            if 'yes' in message_lower:
                response = self.get_prompt('booking_confirmed')
                print(f"Booking to save: {bd}")
                self.context['booking_details'] = {} 
                self.state = 'AWAITING_INITIAL_CHOICE'
            elif 'no' in message_lower:
                response = self.get_prompt('booking_cancelled_user')
                self.context['booking_details'] = {}
                self.state = 'AWAITING_INITIAL_CHOICE'
            else:
                response = "Please answer with Yes or No. " + self.get_prompt('confirm_booking_prompt', city=bd.get('city'), location=bd.get('location'), date=bd.get('date'), time=bd.get('time'), pax_size=bd.get('pax_size'))

        elif self.state == 'COLLECT_CANCEL_MODIFY_DATETIME':
            bd['cancel_datetime_str'] = message
            self.state = 'COLLECT_CANCEL_MODIFY_RESERVATION_NAME'
            response = "And under what name was the reservation made?"

        elif self.state == 'COLLECT_CANCEL_MODIFY_RESERVATION_NAME':
            bd['cancel_name'] = message
            self.state = 'CONFIRM_CANCELLATION'
            response = self.get_prompt('confirm_cancellation_prompt', name=bd.get('cancel_name'), datetime_str=bd.get('cancel_datetime_str'))

        elif self.state == 'CONFIRM_CANCELLATION':
            if 'yes' in message_lower:
                response = self.get_prompt('cancellation_confirmed')
                print(f"Booking to cancel: {bd}")
                self.context['booking_details'] = {}
                self.state = 'AWAITING_INITIAL_CHOICE'
            elif 'no' in message_lower:
                response = self.get_prompt('cancellation_aborted')
                self.context['booking_details'] = {}
                self.state = 'AWAITING_INITIAL_CHOICE'
            else:
                response = "Please answer with Yes or No. " + self.get_prompt('confirm_cancellation_prompt', name=bd.get('cancel_name'), datetime_str=bd.get('cancel_datetime_str'))

        elif self.state == 'HANDLE_ENQUIRY_ASK':
            query = message_lower
            found_info = False
            best_match_text = ""
            best_match_source = ""
            query_keywords = set(query.split())

            if not self.knowledge_base:
                response = "I apologize, my knowledge base is currently unavailable. I cannot answer specific questions right now. Please check the path 'knowledge_base/Knowledge Base/' relative to the notebook."
            else:
                for kb_name, text_content in self.knowledge_base.items():
                    if not text_content: continue
                    content_words = set(text_content.lower().split()) 
                    common_words = query_keywords.intersection(content_words)
                    if len(common_words) > 0:
                        first_keyword = list(common_words)[0]
                        try:
                            idx = text_content.lower().index(first_keyword)
                            snippet = text_content[max(0, idx-50):idx+150] 
                            best_match_text = f"...{snippet}..."
                        except ValueError:
                            best_match_text = "I found some information related to your query."
                        best_match_source = kb_name.replace('_', ' ')
                        found_info = True
                        break 

                if found_info:
                    response = f"Regarding your query about '{query}': From our information for {best_match_source}, I found: {best_match_text} For more details, you might need to check the specific document or our website."
                else:
                    response = f"I couldn't find specific information for '{query}' in my current knowledge base. You might want to check our website or call the outlet directly."
            self.state = 'AWAITING_INITIAL_CHOICE'

        else:
            response = self.get_prompt('master_inform_fallback	') # Corrected key
            self.state = 'AWAITING_INITIAL_CHOICE'

        self.context['booking_details'] = bd
        return response

In [ ]:
# --- Main interaction loop (for testing in a console-like environment) ---
if __name__ == '__main__':
    print("Starting Barbiqnation Chatbot CLI...")
    # Check if directories exist before initializing bot
    if not os.path.isdir(PROMPT_TEMPLATES_DIR):
        print(f"CRITICAL ERROR: Prompt templates directory not found: {os.path.abspath(PROMPT_TEMPLATES_DIR)}")
        print("Please create 'prompt_templates/Prompt Templates/' and place .docx files there.")
    if not os.path.isdir(KNOWLEDGE_BASE_DIR):
        print(f"CRITICAL ERROR: Knowledge base directory not found: {os.path.abspath(KNOWLEDGE_BASE_DIR)}")
        print("Please create 'knowledge_base/Knowledge Base/' and place .pdf files there.")

    bot = BarbiqnationChatbot()
    #print("Bot initialized.")
    print(bot.handle_message("Hi"))

    while True:
        try:
            user_input = input("> " )
            if user_input.lower() == 'quit':
                print("Exiting chatbot. Goodbye!")
                break
            bot_response = bot.handle_message(user_input)
            print(bot_response)
        except EOFError:
            print("Exiting chatbot due to EOF. Goodbye!")
            break
        except KeyboardInterrupt:
            print("Exiting chatbot due to interrupt. Goodbye!")
            break

Starting Barbiqnation Chatbot CLI...
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
pdftotext command not found. Please ensure poppler-utils is installed.
Knowledge base loaded. Keys: ['Bangalore___Bengaluru___Indiranagar', 'Bangalore___Electronic_City', 'Bangalore___JP_Nagar', 'Bangalore___Koramangala_1st_Block', 'Menu_and_Drinks', 'Menu_List', 'New_Delhi___Connaught_Place___CP___cp', 'New_Delhi___Sector_C__Vasant_Kunj', 'New_Delhi___Unity_Mall__Janakpuri']
## OBJECTIVES:
 